In [1]:
import numpy as np
import cv2
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

#
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

###
import prior_box as pb
import tools
from generator import Generator
from data_augmenter import DataAugmenter
from bbox_codec import BBoxCodec

import keras_impl.ssd_pascal as ssd
from keras_impl.losses import SsdLoss



# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
#
img_dir = '../datasets/voc2012/VOCtrainval_11-May-2012/JPEGImages/'
target_img_w, target_img_h = 300, 300
num_classes = 21

lr = 1e-3
batch_size=1
nb_epoch = 10

In [3]:
model = ssd.SSD300(use_bn=False)

In [4]:
#model.summary()
model.compile(optimizer=Adam(lr=lr), loss=SsdLoss(num_classes=num_classes).loss)
#model.compile(optimizer=Adam(lr=lr), loss='mse')

In [5]:
# Load prepared GroundTruth boxes
#gtb = pickle.load(open('data/pascal_voc_2012.p', 'rb'))
gtb = pickle.load(open('data/pascal_voc_2012-with-background.p', 'rb'))

img_file_names = shuffle(list(gtb.keys()))
train_samples, valid_samples = train_test_split(img_file_names, test_size=0.1)

print('Train size: {0}'.format(len(train_samples)))
print('Valid size: {0}'.format(len(valid_samples)))
# test_img_file = '2007_000032.jpg'
# print(gtb[test_img_file].shape)
# print(gtb[test_img_file][0])

Train size: 15282
Valid size: 1698


In [6]:
# Prepare Generator
layers_config = ssd.layers_config
prior_boxes = pb.create_prior_boxes_vect(target_img_w, target_img_h, layers_config, pb.default_prior_variance)
bbox_codec = BBoxCodec(prior_boxes, num_classes)
augmenter = DataAugmenter(target_image_size=(target_img_w, target_img_h))
gen = Generator(gtb, img_dir, augmenter, bbox_codec)

In [9]:
batch_size=8
# compile and train the model using the generator function
train_generator = gen.flow(train_samples, batch_size=batch_size)
valid_generator = gen.flow(valid_samples, batch_size=batch_size)

model_save_path='models/model.h5'
checkpoint = ModelCheckpoint(model_save_path, monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
# tensor_board = TensorBoard(log_dir='./tb-logs', histogram_freq=1, write_graph=True, write_images=True)

nb_samples_per_epoch = len(train_samples)
nb_valid_samples = len(valid_samples)

nb_steps_per_epoch = int(nb_samples_per_epoch/batch_size)
nb_valid_steps_per_epoch = int(nb_valid_samples/batch_size)

print('nb_steps_per_epoch: {0}'.format(nb_steps_per_epoch))

history = model.fit_generator(train_generator, steps_per_epoch=nb_steps_per_epoch, #samples_per_epoch=nb_samples_per_epoch,
                              validation_data=valid_generator, validation_steps=nb_valid_steps_per_epoch, #nb_val_samples=nb_valid_samples,
                              callbacks=[checkpoint, early_stopping],  # , tensor_board],
                              epochs=nb_epoch,
                              )


tools.plot_history_curve(history, "./loss-curve.png")


nb_steps_per_epoch: 2
Epoch 1/10
2/2 [==============================] - 284s - loss: 43.2932 - val_loss: 86.5538
Epoch 2/10
2/2 [==============================] - 287s - loss: 70.7821 - val_loss: 41.7738
Epoch 3/10
2/2 [==============================] - 282s - loss: 41.5927 - val_loss: 42.2579
Epoch 4/10
2/2 [==============================] - 282s - loss: 42.5006 - val_loss: 43.9562
Epoch 5/10
2/2 [==============================] - 282s - loss: 42.9095 - val_loss: 46.4847
Epoch 6/10
2/2 [==============================] - 283s - loss: 42.6629 - val_loss: 41.5741
Epoch 7/10
2/2 [==============================] - 283s - loss: 41.5321 - val_loss: 40.8172
Epoch 8/10
2/2 [==============================] - 283s - loss: 41.2297 - val_loss: 39.3649
Epoch 9/10
2/2 [==============================] - 282s - loss: 38.6621 - val_loss: 42.0528
Epoch 10/10
2/2 [==============================] - 282s - loss: 42.3346 - val_loss: 42.3909


In [ ]:
x_batch, y_batch = next(gen.flow(img_file_names, batch_size=32))
y_pred = model.predict(x_batch)
print('y_pred:{0}'.format(y_pred.shape))

Getting next batch...


In [11]:
import keras.backend as K
import keras.losses as losses
import tensorflow as tf

y_true_keras = K.variable(y_batch)
y_pred_keras = K.variable(y_pred)

loss = SsdLoss(num_classes=num_classes)
loss_val = loss.loss(y_true_keras, y_pred_keras)

y_true_classes = y_true_keras[:, :, 4:4 + num_classes]
y_pred_classes = y_pred_keras[:, :, 4:4 + num_classes]
crossentropy = losses.categorical_crossentropy(y_true_classes, y_pred_classes)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = loss_val.eval()
    #crossentropy_result = crossentropy.eval()
    #true_classes  = y_true_classes.eval()
    #pred_classes  = y_pred_classes.eval()
    
print(result)

444.816
